In [21]:
import numpy as np
import matplotlib.pyplot as plt
import autokeras as ak
import tensorflow as tf
import os
%load_ext tensorboard

In [22]:
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Convolution2D, MaxPooling2D , Lambda, Conv2D, Activation,Concatenate
from tensorflow.keras.layers import ActivityRegularization, BatchNormalization
from tensorflow.keras.optimizers import Adam , SGD , Adagrad
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, CSVLogger, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers , initializers
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import NumpyArrayIterator
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import backend as K

In [23]:
import logging

logging.basicConfig(level = logging.INFO)

gpus = tf.config.experimental.list_physical_devices('GPU')
# if gpus:
#   # Restrict TensorFlow to only use the first GPU
try:
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    tf.config.experimental.set_virtual_device_configuration(
    gpus[0],
    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5000)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
#     logging.info(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
except RuntimeError as e:
# Visible devices must be set before GPUs have been initialized
    logging.info(e)
    
    
logging.info("Tensorflow Version is {}".format(tf.__version__))
logging.info("Keras Version is {}".format(tf.keras.__version__))
from tensorflow.python.client import device_lib
logging.info(device_lib.list_local_devices())
tf.device('/device:XLA_GPU:0')

INFO:root:Tensorflow Version is 2.4.0-rc3
INFO:root:Keras Version is 2.4.0
INFO:root:[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 16444277663331676639
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 5242880000
locality {
  bus_id: 1
  links {
  }
}
incarnation: 11420399614945613284
physical_device_desc: "device: 0, name: TITAN RTX, pci bus id: 0000:03:00.0, compute capability: 7.5"
]


In [24]:
data = np.load('/home/ML4NO/Data/n1000000_0910_classification.npz')

In [25]:
data_all = np.column_stack([data['ve_dune'], data['vu_dune'], data['vebar_dune'], data['vubar_dune']])
target = data['cpv']

x_train = data_all[:10000]
y_train = target[:10000]

x_train2 = data_all[10000:900000]
y_train2 = target[10000:900000]

x_test = data_all[900000:]
y_test = target[900000:]

In [26]:
def DNN_Model(name):
    model_DNN = Sequential(name = "Model_DNN_"+str(name))  
    model_DNN.add(keras.Input(shape=(len(x_train[0]),), name = 'input'))
    model_DNN.add(BatchNormalization(name = 'BatchNormalization'))
    model_DNN.add(Dense(512, activation='relu', name = 'dense_1'))
    model_DNN.add(Dense(1024, activation='relu', name = 'dense_2'))
    model_DNN.add(Dense(1024, activation='relu', name = 'dense_3'))
    model_DNN.add(Dense(1, activation='sigmoid', name = 'output'))
    model_opt = keras.optimizers.Adam()
    model_DNN.compile(loss="binary_crossentropy",
                              optimizer=model_opt,
                              metrics=['accuracy'])
    model_DNN.summary()
    return model_DNN

In [27]:
model = DNN_Model("classification")

Model: "Model_DNN_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
BatchNormalization (BatchNor (None, 264)               1056      
_________________________________________________________________
dense_1 (Dense)              (None, 512)               135680    
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              525312    
_________________________________________________________________
dense_3 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
output (Dense)               (None, 1)                 1025      
Total params: 1,712,673
Trainable params: 1,712,145
Non-trainable params: 528
_________________________________________________________________


In [28]:
index = 1
while os.path.isfile('/home/ML4NO/ML/Classification/models/0910_{}.h5'.format(index)):
    index += 1

In [33]:
log_dir = "logs/{}".format(index)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(x_train2, y_train2,
           validation_split = 0.1,
           callbacks=[tensorboard_callback],
           batch_size=64,
           epochs=20,
           verbose=1,
           shuffle = True
         )

Epoch 1/20
12516/12516 [==============================] - 71s 6ms/step - loss: 0.1853 - accuracy: 0.9363 - val_loss: 0.1819 - val_accuracy: 0.9370
Epoch 2/20
12516/12516 [==============================] - 71s 6ms/step - loss: 0.1838 - accuracy: 0.9364 - val_loss: 0.1656 - val_accuracy: 0.9416
Epoch 3/20
12516/12516 [==============================] - 71s 6ms/step - loss: 0.1832 - accuracy: 0.9367 - val_loss: 0.1507 - val_accuracy: 0.9479
Epoch 4/20
12516/12516 [==============================] - 72s 6ms/step - loss: 0.1819 - accuracy: 0.9371 - val_loss: 0.1593 - val_accuracy: 0.9468
Epoch 5/20
12516/12516 [==============================] - 70s 6ms/step - loss: 0.1808 - accuracy: 0.9383 - val_loss: 0.1457 - val_accuracy: 0.9508
Epoch 6/20
12516/12516 [==============================] - 70s 6ms/step - loss: 0.1808 - accuracy: 0.9380 - val_loss: 0.1616 - val_accuracy: 0.9446
Epoch 7/20
12516/12516 [==============================] - 70s 6ms/step - loss: 0.1804 - accuracy: 0.9382 - val_loss: 0

In [35]:
%tensorboard --logdir logs --bind_all

In [ ]:
model.save('/home/ML4NO/ML/Classification/models/0910_{}.h5'.format(index))

In [22]:
model = tf.keras.models.load_model("/home/ML4NO/ML/Classification/models_all/0910_1.h5")

In [23]:
scale_steps = np.logspace(-3, 0, 10)
before_train_loss = []
after_train_loss = []

for scale in scale_steps:
    x_train2_gen = np.random.normal(x_train2, np.sqrt(x_train2)*scale)
    x_test_gen = np.random.normal(x_test, np.sqrt(x_test)*scale)

    before_train_loss.append(model.evaluate(x_test_gen, y_test)[0])

    model.fit(x_train2_gen, y_train2,
               validation_split = 0.1,
               batch_size=64,
               epochs=5,
               verbose=1,
               shuffle = True
             )

    after_train_loss.append(model.evaluate(x_test_gen, y_test)[0])

3125/3125 [==============================] - 43s 13ms/step - loss: 0.0186 - accuracy: 0.9966
Epoch 1/5
12516/12516 [==============================] - 250s 20ms/step - loss: 0.0209 - accuracy: 0.9957 - val_loss: 0.0259 - val_accuracy: 0.9959
Epoch 2/5
12516/12516 [==============================] - 250s 20ms/step - loss: 0.0207 - accuracy: 0.9957 - val_loss: 0.0194 - val_accuracy: 0.9958
Epoch 3/5
12516/12516 [==============================] - 256s 20ms/step - loss: 0.0206 - accuracy: 0.9958 - val_loss: 0.0278 - val_accuracy: 0.9941
Epoch 4/5
12516/12516 [==============================] - 248s 20ms/step - loss: 0.0199 - accuracy: 0.9960 - val_loss: 0.0298 - val_accuracy: 0.9953
Epoch 5/5
3125/3125 [==============================] - 40s 13ms/step - loss: 0.0154 - accuracy: 0.9968
Epoch 1/5
11146/12516 [=========================>....] - ETA: 24s - loss: 0.0200 - accuracy: 0.9960

KeyboardInterrupt: 

In [ ]:
model_index = index
index = 1
path = '/home/ML4NO/ML/Classification/models_furthurTrain/0910_{}_{}.h5'
while os.path.isfile(path.format(model_index, index)):
    index += 1
model.save(path.format(model_index, index))
outfile = {'scale_steps': scale_steps,
           'before_train_loss': before_train_loss,
           'after_train_loss' :after_train_loss}
np.save(file = '/home/ML4NO/ML/Classification/models_furthurTrain/0910_{}_{}_result.npy'.format(model_index, index),
        arr = outfile)

In [ ]:
x_test2_gen = np.random.poisson(x_test)

for i in range(20):
    x_train2_gen = np.random.poisson(x_train2)
    
    model.fit(x_train2_gen, y_train2,
              validation_split=0.1,
               batch_size=64,
               epochs=1,
               verbose=1,
               shuffle = True
             )
model.evaluate(x_test2_gen, y_test)

In [ ]:
furthur_index = index
index = 1
path = '/home/ML4NO/ML/Classification/models_PoissonTrain/0910_{}_{}_{}.h5'
while os.path.isfile(path.format(model_index, furthur_index, index)):
    index += 1
model.save(path.format(model_index, furthur_index, index))